In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

def get_session(gpu_fraction=0.3):
    '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''
    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

KTF.set_session(get_session(0.5))

Using TensorFlow backend.


In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input
from keras.models import Model

from keras.layers import ZeroPadding2D,Convolution2D
import numpy as np

import os

In [3]:
batch_size = 16
num_classes = 3
epochs = 100

# num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_neuron_classification_trained_model.h5'

In [4]:
# write a function to select x_train y_train x_test y_test
data_x = np.load('image_data_256x256.npy')
data_y = np.load('labels.npy')
data_x = np.swapaxes(np.swapaxes(data_x,1,2),2,3)
data_y = np.array([data_y]).T

from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2)

print x_train.shape, x_test.shape, y_train.shape, y_test.shape
# shuffle the data
# select 20% as the test data and 80% as the training data
# np.random.choice(len(data_x), len(data_x) * 0.2, replace=False)

/Users/EdwardTONG/.pyenv/versions/2.7.13/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(221, 256, 256, 2) (56, 256, 256, 2) (221, 1) (56, 1)


In [5]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print x_train.shape, x_test.shape, y_train.shape, y_test.shape

(221, 256, 256, 2) (56, 256, 256, 2) (221, 3) (56, 3)


In [26]:
# define the model 
# CNN structure
# model = Sequential()
# model.add(Conv2D(8, (3, 3), padding='same',
#                  input_shape=x_train.shape[1:]))
# model.add(Activation('relu'))
# model.add(Conv2D(8, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(16, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(16, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(256))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

img_input = Input(shape = (256, 256, 2))
DIM_ORDERING = "tf"

if DIM_ORDERING == 'tf':
    x_train = np.transpose(x_train.astype('float32') / 255., (0, 1, 2, 3))
    x_test = np.transpose(x_test.astype('float32') / 255., (0, 1, 2, 3))

if DIM_ORDERING == 'th':
    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.

# Layer Cluster - 1
x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(img_input)
# x = Convolution2D(64, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
x = Convolution2D(16, 3, 3,
                  activation='relu',
                  border_mode='same',
                  dim_ordering=DIM_ORDERING)(x)
x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
# x = Convolution2D(64, 3, 3,
#                   activation='relu',
#                   border_mode='same',
#                   dim_ordering=DIM_ORDERING)(x)
x = Convolution2D(16, 3, 3,
                  activation='relu',
                  border_mode='same',
                  dim_ordering=DIM_ORDERING)(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

# Layer Cluster - 2
x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
x = Convolution2D(128, 3, 3,
                  activation='relu',
                  border_mode='same',
                  dim_ordering=DIM_ORDERING)(x)
x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
x = Convolution2D(128, 3, 3,
                  activation='relu',
                  border_mode='same',
                  dim_ordering=DIM_ORDERING)(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

# Layer Cluster - 3
x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
x = Convolution2D(256, 3, 3,
                  activation='relu',
                  border_mode='same',
                  dim_ordering=DIM_ORDERING)(x)
x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
x = Convolution2D(256, 3, 3,
                  activation='relu',
                  border_mode='same',
                  dim_ordering=DIM_ORDERING)(x)
x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
x = Convolution2D(256, 3, 3,
                  activation='relu',
                  border_mode='same',
                  dim_ordering=DIM_ORDERING)(x)
x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
x = Convolution2D(256, 3, 3,
                  activation='relu',
                  border_mode='same',
                  dim_ordering=DIM_ORDERING)(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

# Layer Cluster - 4
x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
x = Convolution2D(512, 3, 3,
                  activation='relu',
                  border_mode='same',
                  dim_ordering=DIM_ORDERING)(x)
x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
x = Convolution2D(512, 3, 3,
                  activation='relu',
                  border_mode='same',
                  dim_ordering=DIM_ORDERING)(x)
x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
x = Convolution2D(512, 3, 3,
                  activation='relu',
                  border_mode='same',
                  dim_ordering=DIM_ORDERING)(x)
x = ZeroPadding2D(padding=(1, 1), dim_ordering=DIM_ORDERING)(x)
x = Convolution2D(512, 3, 3,
                  activation='relu',
                  border_mode='same',
                  dim_ordering=DIM_ORDERING)(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

# Layer Cluster - 5 - Output Layer
x = Flatten()(x)
x = Dense(num_classes, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax')(x)

/Users/EdwardTONG/.pyenv/versions/2.7.13/lib/python2.7/site-packages/ipykernel_launcher.py:38: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(1, 1), data_format="channels_last")`
/Users/EdwardTONG/.pyenv/versions/2.7.13/lib/python2.7/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", activation="relu", data_format="channels_last")`
/Users/EdwardTONG/.pyenv/versions/2.7.13/lib/python2.7/site-packages/ipykernel_launcher.py:47: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(1, 1), data_format="channels_last")`
/Users/EdwardTONG/.pyenv/versions/2.7.13/lib/python2.7/site-packages/ipykernel_launcher.py:55: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", activation="relu", data_format="channels_last")`
/Users/EdwardTONG/.pyenv/versions/2.7.13/lib/python2.7/site-packages/ipykernel

In [27]:
model = Model(inputs = img_input, outputs = x)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 256, 256, 2)       0         
_________________________________________________________________
zero_padding2d_49 (ZeroPaddi (None, 258, 258, 2)       0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 258, 258, 16)      304       
_________________________________________________________________
zero_padding2d_50 (ZeroPaddi (None, 260, 260, 16)      0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 260, 260, 16)      2320      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 130, 130, 16)      0         
_________________________________________________________________
zero_padding2d_51 (ZeroPaddi (None, 132, 132, 16)      0         
__________

In [ ]:
data_augmentation = True

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    for _ in range(epochs):
        model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=1,
                            validation_data=(x_test, y_test))
        model.evaluate(x=x_test, y=y_test, batch_size=batch_size)

Using real-time data augmentation.


/Users/EdwardTONG/.pyenv/versions/2.7.13/lib/python2.7/site-packages/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (221, 256, 256, 2) (2 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')


Epoch 1/1
 2/13 [===>..........................] - ETA: 904s - loss: 1.0979 